In [1]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 91.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.9 MB/s eta 0:00:00


In [2]:
!pip install streamlit -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 114.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 125.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 7.9 MB/s eta 0:00:00


In [9]:
!wget -q -O - ipv4.icanhazip.com

34.143.142.138


In [4]:
!pip install pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 100.2 MB/s eta 0:00:00


In [5]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 67.6 MB/s eta 0:00:00


In [14]:
%%writefile app.py
import os
import streamlit as st
import requests
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter

API_KEY = "given api key"
API_URL = "given api url"
HEADERS = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
}

@st.cache_data(show_spinner=False)
def load_pdf_chunks(pdf_path):
    loader = PyMuPDFLoader(pdf_path)
    documents = loader.load()
    splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    return splitter.split_documents(documents)

@st.cache_resource(show_spinner=False)
def build_vector_store(_chunks):
    embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
    return FAISS.from_documents(_chunks, embedding)
def retrieve_context(vectorstore, query, top_k=3):
    results = vectorstore.similarity_search(query, k=top_k)
    return "\n".join([doc.page_content for doc in results])

def ask_deepseek_llm(context, question): #deepseek model used
    prompt = f"""You are the expert assistant and would help customers with insurance policy questions.
Based on the insurance document attached context, answer the customer's question accurately.
Context:
{context}

User Question: {question}

Answer:"""

    payload = {
        "model": "deepseek/deepseek-chat:free",
        "messages": [{"role": "user", "content": prompt}],
    }

    try:
        response = requests.post(API_URL, headers=HEADERS, json=payload)
        if response.status_code == 200:
            return response.json()["choices"][0]["message"]["content"]
        else:
            return f"API Error {response.status_code}: {response.text}"
    except Exception as e:
        return f"Failed to connect: {e}"

# Streamlit UI
def main():
    st.set_page_config(page_title="Insurance Chatbot", layout="centered")
    st.title("Insurance Policy Chatbot")

    uploaded_file = st.file_uploader("Upload an insurance policy PDF", type="pdf")

    if uploaded_file:
        with open("temp.pdf", "wb") as f:
            f.write(uploaded_file.read())

        st.success("File uploaded.")
        chunks = load_pdf_chunks("temp.pdf")
        vectorstore = build_vector_store(chunks)
        st.success("Document processed!")

        question = st.text_input("Ask your question here:")
        if question:
            with st.spinner("Searching..."):
                context = retrieve_context(vectorstore, question)
                answer = ask_deepseek_llm(context, question)
            st.markdown("### Answer:")
            st.write(answer)

# Run app
if __name__ == "__main__":
    main()


Overwriting app.py


In [ ]:
!streamlit run app.py & npx localtunnel --port=8501